## Prueba #0501

### "Regresión Logísitica y algoritmo de Gradiente Descendente"

#### Responda las siguientes preguntas proporcionando código Python:
#### Objetivos:
- Codifique una clase de regresión logística utilizando solo la biblioteca NumPy.
- Implementar en Python la función Sigmoid.
- Implementar en Python el Gradiente de la probabilidad logarítmica.
- Implementar en Python el algoritmo de gradiente descendente.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
%matplotlib inline
warnings.filterwarnings(action='ignore')                  # Desactiva las advertencias.

#### Leer los datos:

In [2]:
# Cargar los datos
data = load_breast_cancer()
# Variables explicativas
X = data['data']
# Volver a etiquetar para que 0 = 'benign' y 1 = malignant.
Y = 1 - data['target']

In [3]:
# Dividir el conjunto de datos en entrenamiento y prueba.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=1234)

1). Defina las funciones 'sigmoide' y 'gradiente' para producir el resultado que se muestra a continuación:

Función sigmoide:

$$
logit(x) = \frac{1}{1+ e^{-x}}
$$

Descenso de gradiente:

$$
w_j \leftarrow w_j - \eta \left(\frac{1}{n} \sum_{i=1}^{n} x_{i,j} ( logit(wx_i+b)-y_i) \right)
$$

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) 
    
def gradient(X, Y, beta):
    x_por_betas = np.dot(X, beta)
    pred = sigmoid(x_por_betas)
    error_obt = pred - Y
    return (1/Y.size)*np.dot(X.T, error_obt)

2). Defina la clase 'LogisticRegression' para producir el resultado que se muestra a continuación:

In [5]:
class LogisticRegression:
    def __init__(self, learn_rate):
        # learn_rate: Hiperparámetro para el regresor Stochastic Gradient Descent (SGDRegressor)
        self.learning_rate = learn_rate
        self.beta = np.array([])
        self.beta2 = np.array([])
        
    def train(self, input_X, input_Y, n_epochs):
        # inicializar beta
        self.beta = np.zeros(input_X[0,:].size+1)
        self.beta2 = np.zeros(input_X[0,:].size+1)
        # se incluye un 1 a los datos para que la primera columna se multiplique por beta0
        col_1s = np.ones((input_X.shape[0],1))
        X_train_con_beta0 = np.concatenate((col_1s, input_X), axis=1)
        
        # algoritmo de gradiente descendente.
        for iter in range(n_epochs):
            # calcular gradiente
            gradiente = gradient(X_train_con_beta0, input_Y ,self.beta)
            # actualizar valores beta
            self.beta = self.beta - (self.learning_rate * gradiente) 

    def query(self, input_X, prob=True, cutoff=0.5):
        # Multiplicar cada muestra de test por los valores de los coeficientes obtenidos (beta)
        mult_x_betas =  input_X*self.beta[1:self.beta.size]   
        # Sumar los valores por fila (que cada observación de test) y sumarle el intercept (beta0)
        y_val = mult_x_betas.sum(axis=1) + self.beta[0]

        # A los resultados obtenidos, se le aplica la función sigmoid
        y_pred_prob = sigmoid(y_val)
        y_pred = np.array([])

        if prob:
            y_pred = y_pred_prob.reshape(-1,1)
        else:    
            # salida teniendo en cuenta el umbral la probabilidad obtenida y el umbral especificado
            salida_pred_umbral = (y_pred_prob >= cutoff).astype(int)
            y_pred = salida_pred_umbral.reshape(-1,1)
        return y_pred

#### Ejecutar:

In [15]:
# Hiperparámetro para el regresor.
learning_rate = 0.001

In [16]:
# Entrenar y predecir.
LR = LogisticRegression(learning_rate)
LR.train(X_train, Y_train, 2000)
Y_pred = LR.query(X_test,prob=False,cutoff=0.5)

In [17]:
# Mostrar la exactitud
acc = (Y_pred == Y_test.reshape(-1,1)).mean()
print('Accuracy : {}'.format(np.round(acc,3)))
sal_pred, rep_pred = np.unique(Y_pred, return_counts=True)
#print('Datos de la predicción:' ,sal_pred, rep_pred)

Accuracy : 0.908


In [9]:
from sklearn.metrics import classification_report

# y=0 -> Benign, y=1 -> maligno       
# cambiar los nombres de las etiquetas de la salida
list_etiq_invert = list(data['target_names'])
list_etiq_invert.reverse()
etiq_invert = np.array(list_etiq_invert)

# Crea un informe de texto que muestra las principales métricas de clasificación.
print(classification_report(Y_test, Y_pred, target_names=etiq_invert))

              precision    recall  f1-score   support

      benign       0.92      0.93      0.93       144
   malignant       0.88      0.87      0.87        84

    accuracy                           0.91       228
   macro avg       0.90      0.90      0.90       228
weighted avg       0.91      0.91      0.91       228



In [10]:
from sklearn import metrics
# Accuracy obtenido con la librería metrics de sklearn
print( "Accuracy calculado con la librería metrics de sklearn: " + str(metrics.accuracy_score(Y_test,Y_pred)))

Accuracy calculado con la librería metrics de sklearn: 0.9078947368421053


In [11]:
num_pred_err = 0
i = 0
for pred in (Y_pred):
    if pred != Y_test[i]:
        num_pred_err += 1
    i += 1    
print('Número de predicciones erróneas: ', num_pred_err)        

Número de predicciones erróneas:  21


In [12]:
from sklearn import metrics
# Matriz de confusión
# y=0 -> Benign, y=1 -> maligno
matriz_conf = metrics.confusion_matrix(Y_test,Y_pred)
print("Matriz de confusión: ")
print(matriz_conf)

Matriz de confusión: 
[[134  10]
 [ 11  73]]
